<a href="https://colab.research.google.com/github/G544/siburbur/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%5BSC2021%5D_Baseline_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pathlib
import pandas as pd
#from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error

In [4]:
DATA_DIR = pathlib.Path("/content/drive/MyDrive/sibur")
DATA_FILE = "sc2021_train_deals.csv"
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
RS = 82736

# Загрузка данных

In [140]:
data = pd.read_csv(DATA_FILE, parse_dates=["month", "date"])

In [141]:
data.head()

,material_code,company_code,country,region,manager_code,month,material_lvl1_name,material_lvl2_name,material_lvl3_name,contract_type,date,volume
0,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-01,43.0
1,197,0,Китай,Китай,16350,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,95.0
2,794,2162,Казахстан,Атырауская обл.,10942,2018-01-01,Базовые полимеры,ПП,ПП,Контракт,2018-01-02,57.0
3,134,0,Литва,Литва,12261,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,21.0
4,133,0,Китай,Китай,17745,2018-01-01,Базовые полимеры,ПЭ,ПЭНП,Спот,2018-01-02,150.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92306 entries, 0 to 92305
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   material_code       92306 non-null  int64         
 1   company_code        92306 non-null  int64         
 2   country             92306 non-null  object        
 3   region              92306 non-null  object        
 4   manager_code        92306 non-null  int64         
 5   month               92306 non-null  datetime64[ns]
 6   material_lvl1_name  92306 non-null  object        
 7   material_lvl2_name  92306 non-null  object        
 8   material_lvl3_name  92306 non-null  object        
 9   contract_type       92306 non-null  object        
 10  date                92306 non-null  datetime64[ns]
 11  volume              92306 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(6)
memory usage: 8.5+ MB


Временной диапазон тренировочного множества:

In [8]:
data.month.min(), data.month.max()

(Timestamp('2018-01-01 00:00:00'), Timestamp('2020-07-01 00:00:00'))

In [9]:
data.describe()

,material_code,company_code,manager_code,volume
count,92306.000000,92306.000000,92306.000000,92306.000000
mean,430.662752,3736.563831,14589.355687,33.490131
std,237.791293,3283.773053,2412.087205,46.954516
min,124.000000,0.000000,10642.000000,1.000000
25%,214.000000,0.000000,12444.000000,20.000000
50%,443.000000,3342.000000,14676.000000,20.000000
75%,576.000000,6459.000000,16156.000000,32.000000
max,998.000000,9961.000000,19906.000000,1623.000000


In [29]:
all_countries = np.unique(np.array(data['country']), return_counts=True)

In [10]:
len(data['material_code'].unique())

90

# Временные ряды

In [11]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)

In [12]:
group_ts

month                                                                  2018-01-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              340.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301                0.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956               60.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-02-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              340.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942               20.0   
                           Казахстан  г. Нур-Султан      13301                0.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956               60.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-03-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              260.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301               30.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956              100.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-04-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              240.0   
133           0            Белоруссия Минская обл.       10942              200.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301               30.0   
...                                                                           ...   
98

In [13]:
group_ts.index[0]

(124, 7278, 'Россия', 'Респ. Татарстан', 17460)

### Добавим признак число поставок в месяц и регулярность поставок

In [13]:
group_ts1 = data.groupby(AGG_COLS + ["date"])["volume"].sum().unstack(fill_value=0)
#((group_ts1.loc[group_ts1.index[0]]!=0).astype(int).values)
#group_ts1.loc[group_ts1.index[0]].index.month==1

In [14]:
def get_features(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(6)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    features[[f"vol_tm{i}" for i in range(6, 0, -1)]] = df.loc[:, start_period:end_period].copy()

    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    features = features.join(rolling.min().iloc[:, -1].rename("last_year_min"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_year_max"))
    features["month"] = month.month
    return features

In [15]:
tr_range = pd.date_range("2019-01-01", "2019-06-01", freq="MS")
val_range = pd.date_range("2019-07-01", "2019-12-01", freq="MS")
ts_range = pd.date_range("2020-01-01", "2020-07-01", freq="MS")

In [16]:
full_features = {}

for dataset, dataset_range in zip(["tr", "val", "ts"], [tr_range, val_range, ts_range]):
    dataset_features = []
    for target_month in dataset_range:
        features = get_features(group_ts, target_month)
        features["target"] = group_ts[target_month]
        dataset_features.append(features.reset_index())
    full_features[dataset] = pd.concat(dataset_features, ignore_index=True)

In [17]:
full_features['tr']

,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,last_year_min,last_year_max,target
0,124,7278,Россия,Респ. Татарстан,17460,1,220.0,220.0,220.0,280.0,280.0,280.0,260.000000,220.0,340.0,200.0
1,133,0,Белоруссия,Минская обл.,10942,1,0.0,0.0,0.0,0.0,0.0,0.0,21.666667,0.0,200.0,0.0
2,133,0,Белоруссия,Могилевская обл.,10942,1,0.0,0.0,100.0,220.0,20.0,0.0,40.000000,0.0,220.0,0.0
3,133,0,Белоруссия,г. Минск,10942,1,180.0,99.0,60.0,400.0,120.0,20.0,91.583333,0.0,400.0,40.0
4,133,0,Казахстан,г. Нур-Султан,13301,1,40.0,20.0,40.0,30.0,0.0,40.0,19.166667,0.0,40.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,986,9943,Россия,Смоленская обл.,17460,6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5642,998,0,Россия,Ленинградская обл.,18079,6,0.0,5.0,5.0,0.0,8.0,5.0,3.083333,0.0,8.0,10.0
5643,998,3380,Россия,Ленинградская обл.,14956,6,0.0,0.0,0.0,80.0,94.0,127.0,25.083333,0.0,127.0,121.0
5644,998,5410,Россия,г. Санкт-Петербург,14956,6,100.0,120.0,119.0,160.0,120.0,140.0,104.916667,80.0,160.0,100.0


# Закодируем категориальные признаки

In [18]:
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
FTS_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month",
            "vol_tm6", "vol_tm5", "vol_tm4", "vol_tm3", "vol_tm2", "vol_tm1",
            "last_year_avg", "last_year_min", "last_year_max"]
TARGET = "target"

code_feat=full_features['tr'].append( full_features['val'])
code_feat=code_feat.append( full_features['ts'])
code_temp=pd.get_dummies(code_feat[CAT_COLS])
code_feat[code_temp.columns]=code_temp
code_feat.drop(CAT_COLS, axis=1, inplace=True)
code_feat.sample(2)

,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,last_year_min,last_year_max,target,...,region_Чувашская респ.,region_Швейцария,region_Швеция,region_Эстония,region_Ярославская обл.,region_г. Алма-Ата,region_г. Минск,region_г. Москва,region_г. Нур-Султан,region_г. Санкт-Петербург
777,165.0,151.0,107.0,187.0,300.0,170.0,150.5,60.0,300.0,303.0,...,0,0,0,0,0,0,0,0,0,0
2072,42.0,0.0,0.0,0.0,0.0,42.0,7.0,0.0,42.0,63.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
train = code_feat.iloc[0:len(full_features['tr'])+len(full_features['val'])]
test = code_feat.iloc[len(full_features['tr'])+len(full_features['val']):]
y_train = train['target']
X_train = train.drop('target', axis=1)
y_test = test['target']
X_test = test.drop('target', axis=1)

# Обучение

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty = 'elasticnet',solver='saga', C=0.5, l1_ratio = 0.5, verbose=True, n_jobs=-1)
lr.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


max_iter reached after 770 seconds


C:\Programs\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 12.8min finished


LogisticRegression(C=0.5, l1_ratio=0.5, n_jobs=-1, penalty='elasticnet',
                   solver='saga', verbose=True)

In [24]:
preds = lr.predict(X_test)

print("Ошибка на тестовом множестве:",
      f'{np.sqrt(mean_squared_log_error(y_test, preds)):.4f}')

Ошибка на тестовом множестве: 3.2152


In [52]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [27]:
preds = rf.predict(X_test)

print("Ошибка на тестовом множестве:",
      f'{np.sqrt(mean_squared_log_error(y_test, preds)):.4f}')

Ошибка на тестовом множестве: 2.0620


# Случайный лес по странам

In [38]:
all_countries

(array(['Австрия', 'Белоруссия', 'Бельгия', 'Венгрия', 'Германия', 'Дания',
        'Индия', 'Испания', 'Италия', 'Казахстан', 'Киргизия', 'Китай',
        'Литва', 'Молдавия', 'Нидерланды', 'Польша', 'Россия', 'Сербия',
        'Словакия', 'Соед. Королев.', 'Таджикистан', 'Турция',
        'Узбекистан', 'Украина', 'Финляндия', 'Франция', 'Чехия',
        'Швейцария', 'Швеция', 'Эстония'], dtype=object),
 array([  358,  3414,    82,   109,  1602,    42,    44,    17,   157,
         1180,   145,  1871,   223,   281,   279,  1405, 68300,   624,
           72,   107,    47,   363,   127,  4781,   383,  5652,   276,
          182,   160,    23], dtype=int64))

In [135]:
country = ['country_'+s for s in all_countries[0]]
for item in country:
    print(item)
    data = code_feat[code_feat[item]==1]
    split_size = round(data.shape[0]*0.2)
    train = data.iloc[0:split_size]
    test = data.iloc[split_size:]
    y =data['target']
    X = data.drop('target', axis=1)
    y_train = train['target']
    X_train = train.drop('target', axis=1)
    y_test = test['target']
    X_test = test.drop('target', axis=1)
    rf = RandomForestRegressor()
    rf.fit(X, y)
    
    filename = item
    pickle.dump(rf, open(filename, 'wb')) 
    preds = rf.predict(X_test)

    #print("Ошибка на тестовом множестве:",
        #  f'{np.sqrt(mean_squared_log_error(y_test, preds)):.4f}')

country_Австрия
country_Белоруссия
country_Бельгия
country_Венгрия
country_Германия
country_Дания
country_Индия
country_Испания
country_Италия
country_Казахстан
country_Киргизия
country_Китай
country_Литва
country_Молдавия
country_Нидерланды
country_Польша
country_Россия
country_Сербия
country_Словакия
country_Соед. Королев.
country_Таджикистан
country_Турция
country_Узбекистан
country_Украина
country_Финляндия
country_Франция
country_Чехия
country_Швейцария
country_Швеция
country_Эстония


# Формирование загрузки мое

In [55]:
import pickle
filename = 'baseline_model.cbm'
pickle.dump(rf, open(filename, 'wb')) 


In [136]:
%%writefile predict.py

import pathlib
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
from copy import deepcopy
import numpy as np


MODEL_FILE = pathlib.Path(__file__).parent.joinpath("baseline_model.cbm")
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]

def get_features(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:
    """Вычисление признаков для `month`."""

    start_period = month - pd.offsets.MonthBegin(6)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    features[[f"vol_tm{i}" for i in range(6, 0, -1)]] = df.loc[:, start_period:end_period].copy()

    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    features = features.join(rolling.min().iloc[:, -1].rename("last_year_min"))
    features = features.join(rolling.max().iloc[:, -1].rename("last_year_max"))
    features["month"] = month.month
    return features.reset_index()


def predict(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:
    """
    Вычисление предсказаний.

    Параметры:
        df:
          датафрейм, содержащий все сделки с начала тренировочного периода до `month`; типы
          колонок совпадают с типами в ноутбуке `[SC2021] Baseline`,
        month:
          месяц, для которого вычисляются предсказания.

    Результат:
        Датафрейм предсказаний для каждой группы, содержащий колонки:
            - `material_code`, `company_code`, `country`, `region`, `manager_code`,
            - `prediction`.
        Предсказанные значения находятся в колонке `prediction`.
    """

    group_ts = df.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
    features = get_features(group_ts, month)
    code_temp=pd.get_dummies(features[CAT_COLS])
    features[code_temp.columns]=code_temp
    temp = deepcopy(features)
    features.drop(CAT_COLS, axis=1, inplace=True)
    
    model = RandomForestRegressor()
    all_countries = np.unique(np.array(df['country']), return_counts=True)
    country = ['country_'+s for s in all_countries[0]]
    preds_df = temp[AGG_COLS].copy()
    preds_df['prediction'] = np.zeros([preds_df.shape[0],]) 
    for item in country:
        data = features[features[item]==1]
        filename = item
        model = pickle.load(open(pathlib.Path(__file__).parent.joinpath(filename), 'rb')) 
        preds = model.predict(data)
        preds_df.loc[data.index,"prediction"] = preds
    return preds_df
    

Writing predict.py


Протестируем:

In [137]:
%load_ext autoreload

%autoreload 2
import predict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
predict.get_features(group_ts.iloc[:, :-1], pd.Timestamp("2020-07-01"))

,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,last_year_min,last_year_max
0,124,7278,Россия,Респ. Татарстан,17460,7,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,133,0,Белоруссия,Минская обл.,10942,7,17.0,72.0,250.0,394.0,288.0,210.0,179.833333,17.0,394.0
2,133,0,Белоруссия,Могилевская обл.,10942,7,0.0,0.0,0.0,166.0,62.0,0.0,19.000000,0.0,166.0
3,133,0,Белоруссия,г. Минск,10942,7,0.0,0.0,0.0,21.0,0.0,0.0,22.416667,0.0,83.0
4,133,0,Казахстан,г. Нур-Султан,13301,7,0.0,50.0,40.0,0.0,0.0,50.0,30.000000,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,7,0.0,21.0,63.0,125.0,84.0,84.0,31.416667,0.0,125.0
937,998,0,Россия,Ленинградская обл.,18079,7,5.0,5.0,5.0,0.0,3.0,3.0,4.750000,0.0,10.0
938,998,3380,Россия,Ленинградская обл.,14956,7,73.0,74.0,122.0,100.0,15.0,30.0,85.583333,15.0,129.0
939,998,5410,Россия,г. Санкт-Петербург,14956,7,100.0,180.0,180.0,100.0,140.0,40.0,118.333333,40.0,180.0


In [142]:
predict.predict(data[data.month<"2020-07-01"], pd.Timestamp("2020-07-01"))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\79689\\Documents\\Python Scripts\\Sibur\\country_Австрия'

In [55]:
!zip -r baseline_submission.zip baseline_model.cbm predict.py

"zip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
!ls -lh

In [133]:
data[data.month<"2020-07-01"]['volume']

0         43.0
1         95.0
2         57.0
3         21.0
4        150.0
         ...  
88664     50.0
88665     98.0
88666     11.0
88667    100.0
88668     15.0
Name: volume, Length: 88669, dtype: float64